In [1]:
# %run './style_change_detection.py' "./mini-set/" "./mini-val/" "./cc.en.300.vec.gz"

In [2]:
import style_change_detection
from style_change_detection import *

/home/robert/envs/style-change-detection/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /home/robert/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/robert/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
task_2("./train/", "./validation/", "./cc.en.300.vec.gz")

--------downloaded---------
-------read folder -------------
-------read validation -------------
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_3 (Masking)          (None, 47, 5)             0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 47, 256)           137216    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 47, 1)             257       
Total params: 137,473
Trainable params: 137,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
8960/8960 - 129s - loss: 0.0883 - val_loss: 0.0861
Epoch 2/1000
8960/8960 - 123s - loss: 0.0864 - val_loss: 0.0860
Epoch 3/1000
8960/8960 - 129s - loss: 0.0863 - val_loss: 0.0860
Epoch 4/1000
8960/8960 - 129s - loss: 0.0862 - val_loss: 0.0857
Epoch 5/1000
8960/8960 -

'hello'

In [4]:
import gensim

In [5]:
dataset_folder = "./train/"
validation_folder = "./validation/"
embeddings_dict = "./cc.en.300.vec.gz"


if embedding_model == None:
    embedding_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_dict, binary=False, limit=200000)

print("--------downloaded---------")
compl_measures_all, text_ids, labels_all, embedding_all_docs = read_data(dataset_folder, embedding_model)
print("-------read folder -------------")
val_compl_measures, val_text_ids, val_labels_all, val_embedding_all_docs = read_data(validation_folder, embedding_model)

target = 'multi-author'
val_labels = [item[target] for item in val_labels_all]
train_labels = [item[target] for item in labels_all]  # sc = style change

#----------------- complexity measures
padded_compl_measures, padded_labels_style_change, padded_val_x, padded_val_y = padding(compl_measures_all,
                                                                                        train_labels,
                                                                                        val_compl_measures,
                                                                                        val_labels, target)



--------downloaded---------
-------read folder -------------


In [6]:




normalized_compl = normalize(compl_measures_all, axis=2, order=2)
normalied_validation = normalize(val_compl_measures, axis=2, order=2)
eval_scores_compl = get_evaluation(normalized_compl, normalied_validation, padded_val_x, padded_labels_style_change,
                                   scores)

#----------------- embeddings
padded_embeddings, padded_val_x_embeddings, padded_val_y_embeddings = pad_embeddings(embedding_all_docs,
                                                                                     val_embedding_all_docs,
                                                                                     val_labels,
                                                                                     target)

normalized_embeddings = normalize(padded_embeddings, axis=2, order=2)
normalized_val_emb = normalize(padded_val_x_embeddings, axis=2, order=2)
eval_scores_embeddings = get_evaluation(normalized_embeddings, normalized_val_emb, padded_val_y_embeddings,
                                        padded_labels_style_change, scores)

#----------------- combined complexity feats & embeddings
combined_x = np.concatenate((np.array(normalized_embeddings), np.array(normalized_compl)), axis=2)
combined_val = np.concatenate((np.array(normalized_val_emb), np.array(normalied_validation)), axis=2)
combined_scores = get_evaluation(combined_x, combined_val, padded_val_y, padded_labels_style_change, scores)


#----------------- baselines
majority_baseline_scores, random_baseline_acc = baselines(padded_labels_style_change)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 47, 5)             0         
_________________________________________________________________
bidirectional (Bidirectional (None, 47, 256)           137216    
_________________________________________________________________
time_distributed (TimeDistri (None, 47, 1)             257       
Total params: 137,473
Trainable params: 137,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
8960/8960 - 140s - loss: 0.0683 - val_loss: -2.5979e+00
Epoch 2/1000
8960/8960 - 136s - loss: 0.0672 - val_loss: -1.9706e+00
Epoch 3/1000
8960/8960 - 136s - loss: 0.0669 - val_loss: -2.4157e+00
Epoch 4/1000
8960/8960 - 134s - loss: 0.0666 - val_loss: -3.0147e+00
Epoch 5/1000
8960/8960 - 135s - loss: 0.0667 - val_loss: -2.4968e+00
Epoch 6/1000
8960/8960 - 133s - los

In [7]:
#----------------- print results
print("results complexity measures precision, recall, f1, accuracy: " + str(eval_scores_compl))
print("results embeddings precision, recall, f1, accuracy: " + str(eval_scores_embeddings))
print("results combined precision, recall, f1, accuracy: " + str(combined_scores))
print("majority baseline accuracy: " + str(majority_baseline_scores))
print("random baseline accuracy: " + str(random_baseline_acc))

results complexity measures precision, recall, f1, accuracy: [0.9968423609424338, 0.8169331829341119, 0.8979651374808548, 0.9734232522796352]
results embeddings precision, recall, f1, accuracy: [0.9469678568536961, 0.11289030452198252, 0.2017316913311946, 0.12078267477203647]
results combined precision, recall, f1, accuracy: [0.9414622297789653, 0.11246953224745618, 0.20093486206032538, 0.12155205167173253]
majority baseline accuracy: 0.8798442249240122
random baseline accuracy: 0.6177336626139818


In [11]:
from keras.datasets import imdb
(x_train, x_test), (y_train, y_test) = imdb.load_data(num_words=10)

17465344/17464789 [==============================] - 1s 0us/step


In [16]:
len(x_train)

25000